In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
#데이터 준비
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/cleaned_food_recipe_newline.csv')

# 레시피 검색


*   사용자 입력 재료를 통해 만들 수 있는 음식명, 이미지, 재료, 영양정보, 레시피 출력
* 정규화 함수 사용하여 입력 재료 정규화, 정규화된 입력이 '재료정보'열에 포함되어 있으면 정보 출력





In [6]:
#레시피 검색 함수
#정규화, 검색 기능 구현
def normalize_string(s):
  #문자열이 아닌 경우 빈 문자열 반환
  if not isinstance(s, str):
    return ""
  return s.replace(" ", "")

#음식 검색 함수
def find_recipes_by_ingredients(input_ingredients):
  #입력된 재료 정규화
  ingredients_normalized = [normalize_string(ingredient) for ingredient in input_ingredients.split(",")]

  #'재료정보' 열을 정규화하여 검색
  results = df[df['재료정보'].apply(lambda x: all(ingredient in normalize_string(x) for ingredient in ingredients_normalized))]

  #검색 결과 반환
  return results[['메뉴명', '이미지경로(소)', '재료정보', '열량', '탄수화물', '단백질', '지방', '나트륨', '레시피']]


# 예시 입력 재료
input_ingredients = "애호박, 돼지고기"

# 함수 실행 및 결과 출력
results = find_recipes_by_ingredients(input_ingredients)

if not results.empty:
    for index, row in results.iterrows():
        print(f"음식명: {row['메뉴명']}\n이미지: {row['이미지경로(소)']}\n재료: {row['재료정보']}\n영양 정보: 열량-{row['열량']}, 탄수화물-{row['탄수화물']}, 단백질-{row['단백질']}, 지방-{row['지방']}, 나트륨-{row['나트륨']}\n레시피:\n{row['레시피']}\n{'-'*100}")
else:
    print("해당 재료를 사용하는 레시피를 찾을 수 없습니다.")

음식명: 우렁된장소스 배추롤
이미지: http://www.foodsafetykorea.go.kr/uploadimg/cook/10_00636_2.png
재료: 돼지고기(50g), 배춧잎(5장), 부추(30g), 쌀(100g), 당근(20g),
표고버섯(2개), 양파(50g), 애호박(1/2개), 우렁이(100g),
된장(30g)
영양 정보: 열량-326.3, 탄수화물-43.0, 단백질-22.3, 지방-7.2, 나트륨-976.0
레시피:
1. 끓는 물에 배춧잎을 데친다. 
2. 당근, 호박, 표고버섯, 양파는
입자있게 다진다. 
3. 돼지고기를 썰어 불린 쌀, 다진 야채와
함께 밥을 짓는다. 
4. 데친 배춧잎에 지어놓은 밥을 올려
돌돌만다. 
5. 말아 놓은 배춧잎을 부추로 묶는다. 
6. 찜통에 김이 오르면 ?번을 넣고 약 5분
정도 찐다.              
----------------------------------------------------------------------------------------------------
음식명: 인삼떡갈비
이미지: http://www.foodsafetykorea.go.kr/uploadimg/cook/10_00637_2.png
재료: 소고기(100g), 돼지고기(100g), 대추(3알), 양파(30g),
마늘(20g), 배(1/4개), 애호박(1/2개), 단호박(1/4개),
파프리카(50g), 인삼(1뿌리), 양송이(5개), 배추잎(5장),
저염간장(20g), 설탕(20g), 소금(0.2g), 후춧가루(0.01g)
두부(50g)
영양 정보: 열량-253.3, 탄수화물-15.0, 단백질-26.9, 지방-9.5, 나트륨-196.0
레시피:
1. 대추는 돌려 깍아 씨를 제거 하고 곱게
다진다. 
2. 다진 소고기와 돼지고기에 대추와 소금
후춧가루를 넣고 치댄다. 
3. 애호박과 단호박, 파프리카는
입자있게 썰어 ?에 넣고 잘 치댄다. 
4. 양송이는 편으로 썰어 팬에 익히고,
인삼을 뇌두를 자르고 깨끗이 씻어


#콘텐츠 기반 필터링(Content-Based Filtering) 방식을 활용한 추천 모델 구현


*   '재료정보'열의 텍스트 데이터를 전처리해 feature vector 생성: TF-IDF Vectorize를 사용해 각 재료의 중요도 반영한 벡터 생성
*   생성된 feature vector를 바탕으로 음식 간의 코사인 유사도 계산: 추천 생성시 사용


*   특정 음식에 대한 추천을 생성하기 위해 가장 유사한 음식을 찾는 함수 구현






In [ ]:
#추천 모델

In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

#1. 데이터 준비: '재료정보' 열 사용
df = pd.read_csv('/content/drive/MyDrive/cleaned_food_recipe_newline.csv')
data = df['재료정보'].fillna('') #NaN값 처리

#2. 텍스트 데이터 전처리: TF-IDF 벡터라이저 사용

#불용어 지정
korean_stop_words = ['의', '가', '이', '은', '들', '는', '좀', '잘', '걍', '과', '도', '를', '으로', '자', '에', '와', '한', '하다']

# TfidfVectorizer 초기화 (한국어 불용어 리스트 사용)
tfidf_vectorizer = TfidfVectorizer(stop_words=korean_stop_words)
tfidf_matrix = tfidf_vectorizer.fit_transform(df['재료정보'].fillna(''))  # NaN 값을 처리하고 tfidf_matrix 생성

#3. 유사도 계산: 코사인 유사도
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

#4. 추천 함수
def recommend_dishes(dish, cosine_sim = cosine_sim):
  #선택된 음식에 대한 인덱스 찾기
  idx = df[df['메뉴명'] == dish].index[0]

  #모든 음식에 대한 해당 음식의 유사도 스코어 쌍을 리스트로 생성
  sim_scores = list(enumerate(cosine_sim[idx]))

  #유사도에 따라 음식들을 정렬
  sim_scores = sorted(sim_scores, key = lambda x: x[1], reverse = True)

  #가장 유사한 10개의 음식을 가져옴
  sim_scores = sim_scores[1:11]

  #가장 유사한 10개 음식의 인덱스
  dish_indices = [i[0] for i in sim_scores]

  #가장 유사한 10개 음식의 이름 반환
  return df['메뉴명'].iloc[dish_indices]

#추천 예시
recommend_dishes('새우 두부 계란찜')

298              두릅 샐러드
322               감귤 김치
15               칠곡석류국수
297                  컵밥
23              석류 보쌈김치
928    레몬 들깨 소스를 품은 떡꼬치
12        구운채소와 간장레몬 소스
11      버섯구이와 두부타르타르 소스
260           베이컨 가지 말이
20               부대된장찌개
Name: 메뉴명, dtype: object

# 최종 실행 흐름




1.   검색 기능을 통한 초기 레시피 추천: 사용자가 앱에 재료를 입력하면 검색 함수('find_recipes_by_ingredient')를 사용하여 입력된 재료 기반 레시피를 검색하고 추천
2.   클릭한 레시피와 유사한 레시피 추천: 사용자가 검색 결과 중 하나를 클릭하면, 그 레시피와 유사한 레시피를 찾기 위해 머신러닝 기반 추천 함수 사용. 사용자가 클릭한 레시피의 '메뉴명'을 기반으로 머신러닝 기반 추천 함수 호출

※해당 코드는 사용자가 첫 번째 레시피를 클릭했다고 가정하므로 실제 사용자 선택을 처리하기 위한 인터페이스 구현 필요. recommend_dishes 함수를 사용자가 선택한 레시피에 대해 호출하여 처리할 수 있도록 처리 필요





In [9]:
#최종(검색 + 추천)
# 예시 입력 재료
input_ingredients = "애호박, 돼지고기"

# 1단계: 사용자 입력 기반 레시피 검색
results = find_recipes_by_ingredients(input_ingredients)

if not results.empty:
    print("검색된 레시피:\n")
    for index, row in results.iterrows():
        print(f"음식명: {row['메뉴명']}\n이미지: {row['이미지경로(소)']}\n재료: {row['재료정보']}\n영양 정보: 열량-{row['열량']}, 탄수화물-{row['탄수화물']}, 단백질-{row['단백질']}, 지방-{row['지방']}, 나트륨-{row['나트륨']}\n레시피:\n{row['레시피']}\n{'-'*100}\n")
    # 사용자가 선택할 레시피 예시
    selected_dish = results.iloc[0]['메뉴명']  # 첫 번째 검색 결과를 선택한 것으로 가정
else:
    print("해당 재료를 사용하는 레시피를 찾을 수 없습니다.")
    selected_dish = None

# 2단계: 선택된 레시피 기반 유사 레시피 추천
if selected_dish:
    print(f"\n'{selected_dish}'와(과) 유사한 레시피를 추천합니다:\n")
    similar_dishes = recommend_dishes(selected_dish)
    for dish in similar_dishes:
        print(f"- {dish}")

검색된 레시피:

음식명: 우렁된장소스 배추롤
이미지: http://www.foodsafetykorea.go.kr/uploadimg/cook/10_00636_2.png
재료: 돼지고기(50g), 배춧잎(5장), 부추(30g), 쌀(100g), 당근(20g),
표고버섯(2개), 양파(50g), 애호박(1/2개), 우렁이(100g),
된장(30g)
영양 정보: 열량-326.3, 탄수화물-43.0, 단백질-22.3, 지방-7.2, 나트륨-976.0
레시피:
1. 끓는 물에 배춧잎을 데친다. 
2. 당근, 호박, 표고버섯, 양파는
입자있게 다진다. 
3. 돼지고기를 썰어 불린 쌀, 다진 야채와
함께 밥을 짓는다. 
4. 데친 배춧잎에 지어놓은 밥을 올려
돌돌만다. 
5. 말아 놓은 배춧잎을 부추로 묶는다. 
6. 찜통에 김이 오르면 ?번을 넣고 약 5분
정도 찐다.              
----------------------------------------------------------------------------------------------------

음식명: 인삼떡갈비
이미지: http://www.foodsafetykorea.go.kr/uploadimg/cook/10_00637_2.png
재료: 소고기(100g), 돼지고기(100g), 대추(3알), 양파(30g),
마늘(20g), 배(1/4개), 애호박(1/2개), 단호박(1/4개),
파프리카(50g), 인삼(1뿌리), 양송이(5개), 배추잎(5장),
저염간장(20g), 설탕(20g), 소금(0.2g), 후춧가루(0.01g)
두부(50g)
영양 정보: 열량-253.3, 탄수화물-15.0, 단백질-26.9, 지방-9.5, 나트륨-196.0
레시피:
1. 대추는 돌려 깍아 씨를 제거 하고 곱게
다진다. 
2. 다진 소고기와 돼지고기에 대추와 소금
후춧가루를 넣고 치댄다. 
3. 애호박과 단호박, 파프리카는
입자있게 썰어 ?에 넣고 잘 치댄다. 
4. 양송이는 편으로 썰어 팬에 익히고,
인삼을 뇌두를 